# Reinforcement Learning для игры крестиков ноликов

Попробуем применить Reinforcement learnign на простой игре. Наприме: крестики-нолики.

In [1]:
import numpy as np
import pandas as pd
from keras.layers import Dense, Conv2D, Input, Flatten, Dropout, Activation
from keras.models import Sequential
from keras.models import Model
from random import random
import uuid
import os
import copy
from sklearn.model_selection import train_test_split

D:\ProgrammFiles\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Общее описание алгоритма

Пусть у нас есть модель, которая играет случайным образом. Мы проводим N партий между случайными моделями. Тем самым мы генерируем набор данных. Пусть мы находимся в состоянии s, и делаем ход, который переводит нас в состояние s_1. Состояние s_1 мы запоминаем до окончании партии. Пусть в конце партии у игрока 1 есть множество состояний {s_i_1}, а у игрока 2 множество {s_j_2} и предположим, что игрок 1 победил. Тогда будет сформирован набор данных, в которых всем целевым переменным состояния из множества {s_i_1} присвоенно 1, а всем целевым переменным из множества {s_i_2} 0. При ничейном результате можно писать во все состояния 0,5. Затем обучаем модель на этих данных. Теперь проводим партии между этой моделью и предыдущей, тем самым формируя новый набор данных. И та модель которая окажется сильнее переходит в следующий раунд.  
Обобщая, в каждый момент времени у нас есть лучшая модель. Мы обучаем новую модель на данных и проводим партии между ней и лучшей моделью, формируя новые данные и отбирая лучшую модель

In [3]:
file_path = './tic_toc_dataset/'

In [4]:
f = open(file_path + "dataset.csv", "w")

In [5]:
f.close()

# Механика игры

Для начала необходимо реализовать саму механику игры крестики-нолики.  
Класс Tic управляет всей игрой.  
Конструктор данного класса пинимает два параметра типа Player.  
Функция start() инициализирует начало игры (поле 3 на 3 заполненое нулями, кто первый ходит и запускает цикл игры)  
Функция move() запрашивает ход у player, проверяет на окончание игры. В случае окончания игры, вызывается функция окончания у player.

In [6]:
class Tic:    
    def __init__(self, player1, players2):
        self.players = [player1, player2]
        
    def start(self):        
        self.feed = np.zeros([3, 3], dtype=int)      
        self.step_player = 0
        self.end = False
        self.result = ''
        while self.end == False:
            self.move()      
        return self.result
        
    def check_win(self, x, y):
        if self.feed[0][y] == self.feed[1][y] == self.feed[2][y]:
            return True
        if self.feed[x][0] == self.feed[x][1] == self.feed[x][2]:
            return True        
        if x == y and self.feed[0][0] == self.feed[1][1] == self.feed[2][2]:
            return True
        if x + y == 2 and self.feed[0][2] == self.feed[1][1] == self.feed[2][0]:
            return True
        return False   
    
    def check_end(self):
        for i in self.feed:
            for cell in i:
                if cell == 0:
                    return False
        return True
    
    def move(self):        
        i, j = self.players[self.step_player].do_step(self.feed, self.step_player + 1)        
        if self.feed[i][j] == 0:            
            self.feed[i][j] = self.step_player + 1                    
            if self.check_win(i, j):
                self.players[self.step_player].end("win")
                self.result = str(self.step_player)
                self.players[0 if self.step_player == 1 else 1].end("lose")
                self.end = True
                return            
            if self.check_end():
                self.players[self.step_player].end("draw")
                self.result = 'draw'
                self.players[0 if self.step_player == 1 else 1].end("draw")
                self.end = True
                return
            self.step_player = 0 if self.step_player == 1 else 1
            

# Архитектура

Возможная архитектура нейронной сети. Данный код пока что нигде не использутеся.

In [7]:
inp = Input(shape=(3, 3, 3))

conv = Conv2D(9, (3, 3))(inp)

flat = Flatten()(conv)
dense = Dense(18, activation='relu')(flat)
drop = Dropout(0.1)(dense)
out = Dense(1, activation='softmax')(drop)

model = Model(inputs = [inp], outputs = [out])
model.compile(loss = 'mean_squared_error', optimizer='adam', metrics=['mae', 'acc'])

In [8]:
model = Sequential()
model.add(Conv2D(9, (3, 3), padding='same',
                 input_shape=(3, 3, 3)))
model.add(Activation('relu'))
#model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(18))
model.add(Activation('relu'))
#model.add(Dropout(0.1))
model.add(Dense(1))
#model.add(Activation('softmax'))
model.compile(loss = 'mean_squared_error', optimizer='adam')

Опишем класс Player.  
Конструктор данного класса принимает модель. Которая умеет по состояния игры отвечать, на сколько данная ситуация хорошая.  
В классе Player содержится метод do_step(), который вызывается из класса Tic (см. выше)  
В этом методе происходит следующее:  
1. мы смотрим все возможные ходы.
2. состояния, которые получаться после каждого возможного хода мы подаем в модель, которая говорит насколько этот ходы хороший.
3. выбираем и храним лучший ход.
4. все сделанные ходы в данной партии мы храним в self.steps (когда мы делаем ход мы не знаем к чему он привидет)  
Метод end() вызывется, когда закончится партия. В этом методе необходимо будет записать все ходы в некое хранилище с резуьтатом данной партии. 

In [98]:
class Player:
    def __init__(self, model):        
        self.model = model
        self.steps = []
    def do_step(self, feed, number):        
        #top_x = -1
        #top_y = -1
        #top_value = 0
        #top_step = np.zeros((3, 3, 3), dtype=int)        
        values = []
        xs = []
        ys = []
        steps_poss = []       
        for x in range(3):
            for y in range(3):                
                if feed[x][y] != 0:
                    continue
                step = np.zeros((3, 3, 3), dtype=int)
                for i in range(3):
                    for j in range(3):
                        step[0][i][j] = 1 if feed[i][j] == 1 else 0
                        step[1][i][j] = 1 if feed[i][j] == 2 else 0
                        step[2][i][j] = number - 1
                step[0 if number == 1 else 1][x][y] = 1                
                value = self.model.predict(step.reshape(1, 3, 3, 3))[0]                          
                if type(value) != float:                    
                    value = value[0]                
                if value < 0:
                    value = 0            
                print(value)
                values.append(value)
                xs.append(x)
                ys.append(y)
                steps_poss.append(step)
                #if value > top_value:                    
                    #top_value = value
                    #top_x = x
                    #top_y = y
                    #top_step = np.copy(step)                        
        #self.steps.append(top_step)        
        values = np.array(values)
        s = values.sum()
        values = values / s        
        index = np.random.choice(len(values), p = values)
        self.steps.append(np.copy(steps_poss[index]))        
        return xs[index], ys[index]
        #return top_x, top_y
    def end(self, status):
        np_steps = np.reshape(self.steps, (len(self.steps), 27))
        df = pd.DataFrame(np_steps)
        answer = 0
        if status == 'win':
            answer = 1
        if status == 'draw':
            answer = 0.5
        df.insert(27, '27', answer)    
       # print("Dataframe ", status)
        #print (df)
        with open(file_path+'dataset.csv', 'a') as f:
            df.to_csv(f, header=False)        
        f.close()
        self.steps = []

Первая модель самая примитивня: возвращает случайное число.

In [29]:
## model for random
class Model_random:
    def __init__(self):
        pass
    def predict(self, a):
        return [random()]

# Запуск

Пробуем запустить 2-ух игроков со случайными моделями

In [84]:
f = open(file_path + "dataset.csv", "w")
f.close()

In [85]:
player_top = Player(Model_random())
player2 = Player(Model_random())
tic = Tic(player_top, player2)
print('!!')
for x in range(100):
    tic.start() 

!!


In [86]:
for x in range(100):
    print("#_____________________#")
    print(x)
    df = pd.read_csv(file_path + 'dataset.csv', header=None, usecols=[x for x in range(1, 29)])
    if (len(df) > 1000):
        df = df[len(df)-1000: len(df)]
        f = open(file_path + "dataset.csv", "w")
        df.to_csv(f, header=False) 
        f.close()        
  #  print(x)
  #  print('111111')
    gr = df.groupby([x for x in range(1, 28)], as_index=False)
    dataset = gr.agg({28: np.mean})
    y_train_test = dataset.iloc[::, -1::]
    x_train_test = dataset.iloc[::,:-1:]
    x_train_test = x_train_test.values.reshape((len(x_train_test), 3, 3, 3))
    y_train_test = y_train_test.values
    X_train, X_test, y_train, y_test = train_test_split(x_train_test, y_train_test, test_size = 0.2)
    model.compile(loss = 'mean_squared_error', optimizer='adam')
    model.fit(X_train, y_train, batch_size=32, epochs=30, validation_data=(X_test, y_test), shuffle=True)
   # print(x)
   # print('2222222')
    player_new = Player(model)
    score_top = 0.0
    score_new = 0.0
    tic = Tic(player_top, player_new)    
    for i in range(50):        
        res = tic.start()        
        if res == '0':
            score_top += 1
        elif res == '1':
            score_new += 1
        elif res == 'draw':
            score_top += 0.5
            score_new += 0.5
        else:
            assert()
   # print(x)
   # print('3333')        
    player_new = Player(model)
    tic = Tic(player_new, player_top)
    for i in range(50):        
        res = tic.start()
        if res == '1':
            score_top += 1
        elif res == '0':
            score_new += 1
        elif res == 'draw':
            score_top += 0.5
            score_new += 0.5
        else:
            assert()
    #print(x)
    #print('!!!!')
    print('new', score_new, 'top', score_top)
    if score_new > score_top:
        print('Change!!!')        
        player_top = Player(model)

#_____________________#
0
Train on 487 samples, validate on 122 samples
Epoch 1/30
487/487 [==============================] - 1s 3ms/step - loss: 0.1779 - val_loss: 0.1684
Epoch 2/30
487/487 [==============================] - 0s 152us/step - loss: 0.1512 - val_loss: 0.1593
Epoch 3/30
487/487 [==============================] - 0s 129us/step - loss: 0.1382 - val_loss: 0.1631
Epoch 4/30
487/487 [==============================] - 0s 121us/step - loss: 0.1321 - val_loss: 0.1582
Epoch 5/30
487/487 [==============================] - 0s 127us/step - loss: 0.1228 - val_loss: 0.1602
Epoch 6/30
487/487 [==============================] - 0s 129us/step - loss: 0.1159 - val_loss: 0.1572
Epoch 7/30
487/487 [==============================] - 0s 154us/step - loss: 0.1107 - val_loss: 0.1576
Epoch 8/30
487/487 [==============================] - 0s 139us/step - loss: 0.1060 - val_loss: 0.1584
Epoch 9/30
487/487 [==============================] - 0s 127us/step - loss: 0.1021 - val_loss: 0.1577
Epoch 10/30


568/568 [==============================] - 0s 65us/step - loss: 0.0673 - val_loss: 0.1278
Epoch 19/30
568/568 [==============================] - 0s 79us/step - loss: 0.0663 - val_loss: 0.1302
Epoch 20/30
568/568 [==============================] - 0s 76us/step - loss: 0.0653 - val_loss: 0.1297
Epoch 21/30
568/568 [==============================] - 0s 68us/step - loss: 0.0638 - val_loss: 0.1286
Epoch 22/30
568/568 [==============================] - 0s 68us/step - loss: 0.0612 - val_loss: 0.1338
Epoch 23/30
568/568 [==============================] - 0s 68us/step - loss: 0.0617 - val_loss: 0.1298
Epoch 24/30
568/568 [==============================] - 0s 65us/step - loss: 0.0597 - val_loss: 0.1298
Epoch 25/30
568/568 [==============================] - 0s 68us/step - loss: 0.0582 - val_loss: 0.1302
Epoch 26/30
568/568 [==============================] - 0s 79us/step - loss: 0.0576 - val_loss: 0.1321
Epoch 27/30
568/568 [==============================] - 0s 79us/step - loss: 0.0564 - val_loss:

577/577 [==============================] - 0s 69us/step - loss: 0.1135 - val_loss: 0.1387
Epoch 6/30
577/577 [==============================] - 0s 71us/step - loss: 0.1086 - val_loss: 0.1366
Epoch 7/30
577/577 [==============================] - 0s 74us/step - loss: 0.1051 - val_loss: 0.1396
Epoch 8/30
577/577 [==============================] - 0s 72us/step - loss: 0.1029 - val_loss: 0.1384
Epoch 9/30
577/577 [==============================] - 0s 72us/step - loss: 0.1039 - val_loss: 0.1380
Epoch 10/30
577/577 [==============================] - 0s 63us/step - loss: 0.0992 - val_loss: 0.1447
Epoch 11/30
577/577 [==============================] - 0s 73us/step - loss: 0.0940 - val_loss: 0.1395
Epoch 12/30
577/577 [==============================] - 0s 83us/step - loss: 0.0927 - val_loss: 0.1370
Epoch 13/30
577/577 [==============================] - 0s 85us/step - loss: 0.0885 - val_loss: 0.1386
Epoch 14/30
577/577 [==============================] - 0s 71us/step - loss: 0.0863 - val_loss: 0.1

Epoch 23/30
589/589 [==============================] - 0s 168us/step - loss: 0.0667 - val_loss: 0.1293
Epoch 24/30
589/589 [==============================] - 0s 185us/step - loss: 0.0646 - val_loss: 0.1317
Epoch 25/30
589/589 [==============================] - 0s 173us/step - loss: 0.0635 - val_loss: 0.1318
Epoch 26/30
589/589 [==============================] - 0s 174us/step - loss: 0.0627 - val_loss: 0.1342
Epoch 27/30
589/589 [==============================] - 0s 156us/step - loss: 0.0620 - val_loss: 0.1339
Epoch 28/30
589/589 [==============================] - 0s 144us/step - loss: 0.0609 - val_loss: 0.1361
Epoch 29/30
589/589 [==============================] - 0s 149us/step - loss: 0.0608 - val_loss: 0.1368
Epoch 30/30
589/589 [==============================] - 0s 157us/step - loss: 0.0592 - val_loss: 0.1382
new 40.0 top 60.0
#_____________________#
8
Train on 570 samples, validate on 143 samples
Epoch 1/30
570/570 [==============================] - 3s 5ms/step - loss: 0.1643 - val

565/565 [==============================] - 0s 152us/step - loss: 0.0919 - val_loss: 0.1425
Epoch 11/30
565/565 [==============================] - 0s 122us/step - loss: 0.0891 - val_loss: 0.1408
Epoch 12/30
565/565 [==============================] - 0s 67us/step - loss: 0.0863 - val_loss: 0.1412
Epoch 13/30
565/565 [==============================] - 0s 105us/step - loss: 0.0851 - val_loss: 0.1434
Epoch 14/30
565/565 [==============================] - 0s 94us/step - loss: 0.0824 - val_loss: 0.1450
Epoch 15/30
565/565 [==============================] - 0s 79us/step - loss: 0.0810 - val_loss: 0.1398
Epoch 16/30
565/565 [==============================] - 0s 81us/step - loss: 0.0793 - val_loss: 0.1446
Epoch 17/30
565/565 [==============================] - 0s 69us/step - loss: 0.0773 - val_loss: 0.1418
Epoch 18/30
565/565 [==============================] - 0s 85us/step - loss: 0.0766 - val_loss: 0.1455
Epoch 19/30
565/565 [==============================] - 0s 85us/step - loss: 0.0737 - val_lo

...

580/580 [==============================] - 0s 177us/step - loss: 0.1101 - val_loss: 0.1644
Epoch 6/30
580/580 [==============================] - 0s 169us/step - loss: 0.1088 - val_loss: 0.1650
Epoch 7/30
580/580 [==============================] - 0s 215us/step - loss: 0.1053 - val_loss: 0.1643
Epoch 8/30
580/580 [==============================] - 0s 300us/step - loss: 0.1019 - val_loss: 0.1642
Epoch 9/30
580/580 [==============================] - 0s 364us/step - loss: 0.0998 - val_loss: 0.1637
Epoch 10/30
580/580 [==============================] - 0s 312us/step - loss: 0.0978 - val_loss: 0.1618
Epoch 11/30
580/580 [==============================] - 0s 315us/step - loss: 0.0961 - val_loss: 0.1618
Epoch 12/30
580/580 [==============================] - 0s 287us/step - loss: 0.0962 - val_loss: 0.1653
Epoch 13/30
580/580 [==============================] - 0s 290us/step - loss: 0.0938 - val_loss: 0.1617
Epoch 14/30
580/580 [==============================] - 0s 373us/step - loss: 0.0914 - val

576/576 [==============================] - 0s 192us/step - loss: 0.0670 - val_loss: 0.1621
Epoch 24/30
576/576 [==============================] - 0s 184us/step - loss: 0.0664 - val_loss: 0.1624
Epoch 25/30
576/576 [==============================] - 0s 347us/step - loss: 0.0644 - val_loss: 0.1630
Epoch 26/30
576/576 [==============================] - 0s 259us/step - loss: 0.0631 - val_loss: 0.1655
Epoch 27/30
576/576 [==============================] - 0s 307us/step - loss: 0.0622 - val_loss: 0.1669
Epoch 28/30
576/576 [==============================] - 0s 299us/step - loss: 0.0609 - val_loss: 0.1658
Epoch 29/30
576/576 [==============================] - 0s 301us/step - loss: 0.0602 - val_loss: 0.1665
Epoch 30/30
576/576 [==============================] - 0s 298us/step - loss: 0.0595 - val_loss: 0.1685
new 50.0 top 50.0
#_____________________#
91
Train on 578 samples, validate on 145 samples
Epoch 1/30
578/578 [==============================] - 6s 10ms/step - loss: 0.1263 - val_loss: 0.1

Epoch 10/30
592/592 [==============================] - 0s 120us/step - loss: 0.1062 - val_loss: 0.1344
Epoch 11/30
592/592 [==============================] - 0s 168us/step - loss: 0.1038 - val_loss: 0.1368
Epoch 12/30
592/592 [==============================] - 0s 190us/step - loss: 0.1019 - val_loss: 0.1351
Epoch 13/30
592/592 [==============================] - 0s 225us/step - loss: 0.1006 - val_loss: 0.1334
Epoch 14/30
592/592 [==============================] - 0s 228us/step - loss: 0.0981 - val_loss: 0.1365
Epoch 15/30
592/592 [==============================] - 0s 228us/step - loss: 0.0967 - val_loss: 0.1344
Epoch 16/30
592/592 [==============================] - 0s 237us/step - loss: 0.0958 - val_loss: 0.1337
Epoch 17/30
592/592 [==============================] - 0s 204us/step - loss: 0.0929 - val_loss: 0.1343
Epoch 18/30
592/592 [==============================] - 0s 228us/step - loss: 0.0908 - val_loss: 0.1330
Epoch 19/30
592/592 [==============================] - 0s 247us/step - lo

Epoch 28/30
589/589 [==============================] - 0s 129us/step - loss: 0.0587 - val_loss: 0.1146
Epoch 29/30
589/589 [==============================] - 0s 124us/step - loss: 0.0578 - val_loss: 0.1152
Epoch 30/30
589/589 [==============================] - 0s 134us/step - loss: 0.0574 - val_loss: 0.1165
new 46.0 top 54.0
#_____________________#
96
Train on 579 samples, validate on 145 samples
Epoch 1/30
579/579 [==============================] - 5s 8ms/step - loss: 0.1306 - val_loss: 0.1060
Epoch 2/30
579/579 [==============================] - 0s 153us/step - loss: 0.1107 - val_loss: 0.1021
Epoch 3/30
579/579 [==============================] - 0s 107us/step - loss: 0.1028 - val_loss: 0.1015
Epoch 4/30
579/579 [==============================] - 0s 143us/step - loss: 0.0959 - val_loss: 0.0978
Epoch 5/30
579/579 [==============================] - 0s 138us/step - loss: 0.0909 - val_loss: 0.0995
Epoch 6/30
579/579 [==============================] - ETA: 0s - loss: 0.092 - 0s 136us/step 

Epoch 15/30
590/590 [==============================] - 0s 208us/step - loss: 0.0889 - val_loss: 0.1377
Epoch 16/30
590/590 [==============================] - 0s 225us/step - loss: 0.0872 - val_loss: 0.1375
Epoch 17/30
590/590 [==============================] - 0s 251us/step - loss: 0.0856 - val_loss: 0.1395
Epoch 18/30
590/590 [==============================] - 0s 223us/step - loss: 0.0841 - val_loss: 0.1381
Epoch 19/30
590/590 [==============================] - 0s 204us/step - loss: 0.0827 - val_loss: 0.1384
Epoch 20/30
590/590 [==============================] - 0s 250us/step - loss: 0.0822 - val_loss: 0.1375
Epoch 21/30
590/590 [==============================] - 0s 270us/step - loss: 0.0808 - val_loss: 0.1395
Epoch 22/30
590/590 [==============================] - 0s 239us/step - loss: 0.0799 - val_loss: 0.1382
Epoch 23/30
590/590 [==============================] - 0s 251us/step - loss: 0.0786 - val_loss: 0.1391
Epoch 24/30
590/590 [==============================] - 0s 193us/step - lo

In [ ]:
player_new.do_step

In [101]:
test = np.array([[[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[1, 1, 1], [1, 1, 1], [1, 1, 1]]])

In [102]:
test.shape

(3, 3, 3)

In [103]:
model.predict(test.reshape((1, 3, 3, 3)))[0]

array([0.5213954], dtype=float32)

In [ ]:
df[1510:1511]

In [104]:
player_test = Player(model)

In [115]:
player_test.do_step(np.array([[0, 2, 0], [0, 1, 0], [0, 0, 1]]), 2)

0.18290359
0.21989563
0.24626917
0.17521888
0.06542116
0.0680423


(0, 2)

In [99]:
player_top.do_step(np.array([[2, 1, 1], [0, 1, 0], [0, 0, 0]]), 0)



(1, 0)

In [12]:
df = pd.read_csv(file_path + 'dataset.csv', header=None, usecols=[x for x in range(1, 29)])

In [13]:
gr = df.groupby([x for x in range(1, 28)], as_index=False)
res = gr.agg({28: np.mean})

In [14]:
res.shape

(573, 28)

In [15]:
y_train_test = res.iloc[::, -1::]
x_train_test = res.iloc[::,:-1:]
x_train_test = x_train_test.values.reshape((len(x_train_test), 3, 3, 3))
y_train_test = y_train_test.values
X_train, X_test, y_train, y_test = train_test_split(x_train_test, y_train_test, test_size = 0.2)
model.fit(X_train, y_train, batch_size=32, epochs=30, validation_data=(X_test, y_test), shuffle=True)

Train on 458 samples, validate on 115 samples
Epoch 1/30
458/458 [==============================] - 1s 1ms/step - loss: 0.4352 - val_loss: 0.3165
Epoch 2/30
458/458 [==============================] - 0s 96us/step - loss: 0.2700 - val_loss: 0.2381
Epoch 3/30
458/458 [==============================] - 0s 91us/step - loss: 0.2167 - val_loss: 0.1900
Epoch 4/30
458/458 [==============================] - 0s 109us/step - loss: 0.1873 - val_loss: 0.1743
Epoch 5/30
458/458 [==============================] - 0s 105us/step - loss: 0.1763 - val_loss: 0.1696
Epoch 6/30
458/458 [==============================] - 0s 106us/step - loss: 0.1697 - val_loss: 0.1695
Epoch 7/30
458/458 [==============================] - 0s 103us/step - loss: 0.1642 - val_loss: 0.1699
Epoch 8/30
458/458 [==============================] - 0s 104us/step - loss: 0.1594 - val_loss: 0.1694
Epoch 9/30
458/458 [==============================] - 0s 96us/step - loss: 0.1557 - val_loss: 0.1706
Epoch 10/30
458/458 [====================

In [236]:
model.get_weights()

[array([[[[-2.02421695e-01,  1.71106726e-01,  7.09197521e-02,
           -2.08580345e-01,  1.68643087e-01,  2.55949795e-03,
            7.18033314e-03,  5.76716065e-02],
          [ 2.38285214e-01,  2.23411620e-02, -2.05493122e-01,
           -1.36853307e-01, -2.28690282e-01,  1.33575916e-01,
           -1.96038485e-01, -1.67377800e-01],
          [-2.03690737e-01,  1.12392634e-01, -6.40386790e-02,
            6.71531260e-02,  6.09028041e-02, -1.45236015e-01,
            3.77266109e-03, -1.96479574e-01]],
 
         [[-7.25624859e-02,  1.74175799e-01,  1.73950344e-01,
            1.91863239e-01, -1.52105868e-01, -1.63056284e-01,
            8.97207260e-02,  2.07896739e-01],
          [ 1.05400473e-01,  2.15014964e-01,  1.72521979e-01,
           -3.34192365e-02,  1.77648067e-02,  4.93142009e-02,
           -1.47560447e-01,  2.52798200e-03],
          [ 5.40877879e-02,  3.82365286e-02, -3.69710177e-02,
            4.13408875e-03, -4.18183804e-02, -2.40707710e-01,
            1.39208078e

In [273]:
a = np.zeros((3, 3, 3))
a[0][0][0] = 0
a[0][0][1] = 1
a[0][0][2] = 0
a[0][1][0] = 0
a[0][1][1] = 0
a[0][1][2] = 0
a[0][2][0] = 0
a[0][2][1] = 1
a[0][2][2] = 0
a[1][0][0] = 0
a[1][0][1] = 0
a[1][0][2] = 1
a[1][1][0] = 0
a[1][1][1] = 0
a[1][1][2] = 0
a[1][2][0] = 0
a[1][2][1] = 0
a[1][2][2] = 0
a[2][0][0] = 0
a[2][0][1] = 0
a[2][0][2] = 0
a[2][1][0] = 0
a[2][1][1] = 0
a[2][1][2] = 0
a[2][2][0] = 0
a[2][2][1] = 0
a[2][2][2] = 0

In [274]:
model.predict(a.reshape((1, 3, 3, 3)))[0][0]

0.55212593

In [220]:
model.predict(X_test)

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32)

In [135]:
a.reshape((1, 3, 3, 3))

array([[[[1., 1., 1.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 1., 1.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]])